In [1]:
#Importing python libraries from local files. 
from utils.CameraUtils import *
from utils.MachineUtils import *
import json

In [2]:
#Establish machine communication 
port = '/dev/ttyACM0' #Check options using  - print([port.name for port in serial.tools.list_ports.comports()])
m = MachineCommunication(port)

In [22]:
#Open a live video window
# find the z height at which we can see a whole petri dish
# at z=110 with camera tool equipped, whole first plate is visible 
%matplotlib inline
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

(320, 240)


In [ ]:
f = getFrame()

In [14]:
showFrame(getFrame(), grid=True)

In [5]:
# read in the calibration file
with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_60_tape.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

[[ 2.83177792e-01 -1.52038853e+00]
 [ 2.96527139e+00 -3.34144830e+00]
 [-4.89224219e-01  2.59932583e-01]
 [ 5.55483812e-01  7.52317823e+01]
 [-7.49778260e+01 -4.26956240e-02]
 [ 8.02453667e+01  2.39263142e+02]]
[1200, 1200]


In [6]:
m.transform = matrix
m.img_size = size

In [7]:
m.moveTo(z=60)

In [ ]:
m.moveTo(x=90, y=230)

In [15]:
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

In [ ]:
showFrame(getFrame(), grid=True)

In [16]:
#px_to_real(749, 209, absolute=True)
off = px_to_real(651, 900)
print(off)

[-18.54020549   2.97785191]


In [12]:
o = [90, 250]

In [13]:
m.moveTo(x=o[0], y=o[1])

In [17]:
off = px_to_real(650, 900)
print(off)

[-18.54058634   2.91521124]


In [18]:
p = [o[0] - off[0], o[1] - off[1]]
print(p)

[108.54058633664744, 247.08478875929072]


In [19]:
m.moveTo(x=p[0], y=p[1])

In [32]:
# switch tools and see if it's aligned
m.toolChange(2)

In [ ]:
syringe_off = [-7.9, 5.8]

In [ ]:
m.moveTo(m.moveTo(x=p[0] + syringe_off[0], y=p[1] + syringe_off[1]))

In [20]:
showFrame(getFrame(), grid=True)

In [ ]:
showFrame(getFrameCamera(1), grid=True)

In [42]:
circle, data = getSingleWell(getFrame(), minR = 270, maxR=290)

I used a gain of 1.3000000000000003
the radius is: 283
the center is: (603,600)
I found 1 wells


In [43]:
showFrame(circle)